<a href="https://colab.research.google.com/github/marcostark/Learning-Data-Science/blob/master/desafios/Sentiment_Analysis_with_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Sentimento utilizando Regressão Logística

## Introdução

### Sobre o conjunto de dados

- O dataset utilizado será do IMDB contendo varias avaliações de filmes, está disponível em: http://ai.stanford.edu/~amaas/data/sentiment

  - O dataset contem 25000 avaliações positivas(label=1) e 25000 avaliações negativas(label=0)
  - O conjuto de dados possui apenas duas colunas: review(avaliação) e sentiment(sentimento)


### Modelo de regressão logística

### Importando as bibliotecas necessárias para o projeto
---

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from nltk.stem.porter import PorterStemmer
import pickle


In [0]:
# Carregando o dataset
dataset_uri = "https://raw.githubusercontent.com/marcostark/Learning-Data-Science/master/desafios/datasets/imdb_movie_data.csv"

df_movies = pd.read_csv(dataset_uri)
df_movies.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [0]:
# Total de avalições
print(df_movies.shape)

(50000, 2)


In [0]:
df_movies.columns

Index(['review', 'sentiment'], dtype='object')

In [0]:
# Labels do dataset: (label=1), Negativo(label=0)
df_movies.set_index(['review', 'sentiment']).count(level='sentiment')

""
sentiment
0
1


In [0]:
# Número de labels que representam sentimento positivo
df_movies[df_movies.sentiment==1].count()

review       25000
sentiment    25000
dtype: int64

In [0]:
# Número de labels que representam sentimento negativo
df_movies[df_movies.sentiment==0].count()

review       25000
sentiment    25000
dtype: int64

## Transformando documentos em vetores

In [0]:
count = CountVectorizer()

docs = np.array(['The sun is shinnig',
                 'The weather is sweet',
                 'The sun if shinning, the weather is sweet and one and one is two'])

bag = count.fit_transform(docs)

In [0]:
print(bag.toarray())

[[0 0 1 0 1 0 1 0 1 0 0]
 [0 0 1 0 0 0 0 1 1 0 1]
 [2 1 2 2 0 1 1 1 2 1 1]]


In [0]:
term = df_movies.loc[0,'review'][-50:]
term

'zation my vote is seven title brazil not available'

## Preparação dos dados

In [0]:
import re

def preprocessor(text):
  text = re.sub('<[^>]*>','',text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
  text = re.sub('[\W]+',' ', text.lower()) +\
    ' '.join(emoticons).replace('-', '')
  return text

In [0]:
print(preprocessor(term))

zation my vote is seven title brazil not available


In [0]:
print(preprocessor("</a>This ;) is a :( test :-)!"))

this is a test ;) :( :)


In [0]:
df_movies['review'] = df_movies['review'].apply(preprocessor)

## Etapa de tokenização dos dados

- Consiste oo processo que divide uma sentença em unidades mais básicas

In [0]:
porter = PorterStemmer()

def tokenizer(text):
  return text.split()

def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

In [0]:
tokenizer('Luminous beings are we. Not this crude matter.')

['Luminous', 'beings', 'are', 'we.', 'Not', 'this', 'crude', 'matter.']

In [0]:
tokenizer_porter('Luminous beings are we. Not this crude matter.')

['lumin', 'be', 'are', 'we.', 'not', 'thi', 'crude', 'matter.']

In [0]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('Luminous beings are we. Not this crude matter.')[-10:] if w not in stop]

['lumin', 'we.', 'thi', 'crude', 'matter.']

## Transformando documentos em vetores TF-IDF

- TF-IDF (Term Frequency - Inverse Document Frequency) - utilizado para diminuir
a importância das palabreas exibidas em muitos documentos em comum, que são consideradas de discernir os documentos, em vez de simplesmente contas a frequência das oalavras, como é feito com o CountVectorizer.

In [0]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                        tokenizer = tokenizer_porter,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True)

y = df_movies.sentiment.values
X = tfidf.fit_transform(df_movies.review)

## Classficando documentos utilizando modelo de regressão logística

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.5, shuffle=False)

In [0]:
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(
    cv=5,
    scoring='accuracy',
    random_state=0,
    n_jobs=-1,
    verbose=3,
    max_iter=300).fit(X_train, y_train)

## Salvando modelo em um arquivo
save_model = open('sentiment_analysis_model.sav', 'wb')
pickle.dump(clf, save_model)
save_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.4min finished


## Carregando arquivo e verificação precisão do modelo

In [0]:
file_model = 'sentiment_analysis_model.sav'

save_clf = pickle.load(open(file_model,'rb'))

In [0]:
print('Precisão do teste: {:.3f}'.format(save_clf.score(X_test, y_test)))

Precisão do teste: 0.896
